In [1]:
#https://www.kaggle.com/mousehead/songlyrics/version/1#
#http://warmspringwinds.github.io/pytorch/rnns/2018/01/27/learning-to-generate-lyrics-and-music-with-recurrent-neural-networks/

import os
import numpy as np
from keras.models import Sequential, Model
from keras.layers.recurrent import LSTM, SimpleRNN, GRU
from keras.layers import Activation, BatchNormalization, TimeDistributed, Dense, RepeatVector, recurrent, Embedding,Dropout, Bidirectional, Input, merge, Flatten, Reshape
from keras.preprocessing.sequence import pad_sequences
from keras.optimizers import SGD, Adam, RMSprop
import keras
import json
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_vocab(file_path):
    f = open(file_path, 'r') 
    vocab_dict = json.loads(f.read() )
    f.close() 
    idx_dict = {idx:word for word, idx in vocab_dict.items()}
    return vocab_dict, idx_dict

vocab_to_idx, idx_to_vocab = load_vocab("vocab/vocab_dict.json")
embeding_matrix = np.load("vocab/embeding_matrix.npy")

In [3]:

def create_old_model(max_len, vocab_size, embedding_size, embeding_matrix, hidden_size):
    model = Sequential()
    inputs = Input(shape = (max_len,))
    embedding = Embedding(vocab_size, embedding_size, weights = [embeding_matrix], input_length=max_len, trainable = False)(inputs)
    gru1 = GRU(hidden_size, return_sequences=True)(embedding)
    dropout1 = Dropout(0.2)(gru1)
    gru2 = GRU(hidden_size)(dropout1)
    dropout2 = Dropout(0.2)(gru2)
    dense = Dense(vocab_size)(dropout2)
    activation_output = Activation('softmax')(dense)
    model = Model(inputs = inputs, outputs = activation_output)
    return model
def create_model(max_len, vocab_size, embedding_size, embeding_matrix, hidden_size):
    model = Sequential()
    inputs = Input(shape = (max_len,))
    embedding = Embedding(vocab_size, embedding_size, weights = [embeding_matrix], input_length=max_len, trainable = False)(inputs)
    batch_norm1 = BatchNormalization()(embedding)
    gru1 = GRU(hidden_size, return_sequences=True)(batch_norm1)
    dropout1 = Dropout(0.2)(gru1)
    gru2 = GRU(hidden_size)(dropout1)
    dropout2 = Dropout(0.2)(gru2)
    dense = Dense(vocab_size)(dropout2)
    batch_norm2 = BatchNormalization()(dense)
    activation_output = Activation('softmax')(dense)
    model = Model(inputs = inputs, outputs = activation_output)
    return model

In [4]:
def read_sentences(file_path, column_name, start_row, num_rows):
    if file_path.endswith('.csv') or file_path.endswith('.CSV'):
        df = pd.read_csv(file_path)
        data = []
        data_chunk = df[start_row : start_row + num_rows ]
        for _, row in data_chunk.iterrows():
            line = row[column_name].lower().replace('\n', '').replace('(', '').replace(')', '').replace('  ', ' ')
            data.append(line)
        return data



In [5]:
def preprocess_sentences(sentences):
    abb_word = ['won’t','won\'t','wouldn’t','wouldn\'t','’m', '’re', '’ve', '’ll', '’s','’d', 'n’t', '\'m', '\'re', '\'ve', '\'ll', '\'s', '\'d', 'can\'t', 'n\'t', 'B: ', 'A: ', ',', ';', '.', '?', '!', ':', '. ?', ',   .', '. ,', 'eos', 'bos']
    orig_word = ['will not','will not','would not','would not',' am', ' are', ' have', ' will', ' is', ' had', ' not', ' am', ' are', ' have', ' will', ' is', ' had', 'can not', ' not', '', '', ' ,', ' ;', ' .', ' ?', ' !', ' :', '? ', '.', ',', '', '']
    for i in range(len(sentences)):
        content = sentences[i]
        for idx in range(len(abb_word)):
            content = content.replace(abb_word[idx], orig_word[idx])
        sentences[i] = content
    return sentences


In [6]:
def convert_sentences_to_vectors(sentences, vocab_to_idx):
    vectors = []
    for sen in sentences:
        words = sen.split()
        vectors.append([ vocab_to_idx[w] for w in words if w in vocab_to_idx])
    return vectors


In [7]:
def to_one_hot(vec, sentence_length, vocab_length):
    print((len(vec), vocab_length))
    res = np.zeros((len(vec), vocab_length))
    for i, value in enumerate(vec):
            res[i, value] = 1
    return res

def create_trainning_data(sentence_vectors, window_size, space_idx, vocab_size):
    x = []
    y = []
    for words in sentence_vectors:
        wl = len(words)
        if  wl<= window_size:
                x.append(words + [0] * (window_size - wl)  )
                y.append(0)
        else:
            for start in range(wl - window_size):
                x.append(words[start : start + window_size])
                y.append(words[start + window_size])
    
    return np.array(x), np.array(y)


In [8]:
model = create_model(10 , 30000, 300 , embeding_matrix , 500)
model.summary()
opt = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 300)           9000000   
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 300)           1200      
_________________________________________________________________
gru_1 (GRU)                  (None, 10, 500)           1201500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 500)           0         
_________________________________________________________________
gru_2 (GRU)                  (None, 500)               1501500   
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
__________

In [9]:
model.load_weights("weights/0model.h5")

In [ ]:
def get_trainning_data(start, amount):
    sentences = read_sentences("data\songdata.csv", 'text', start, amount)
    sentences =  preprocess_sentences(sentences)
    print(len(sentences))
    sentence_vectors = convert_sentences_to_vectors(sentences, vocab_to_idx)
    x, y = create_trainning_data(sentence_vectors, 10, ' ', 30000)
    y = to_one_hot(y, 10, 30000)
    return x, y

In [ ]:
i = 0
start = 100
amount = 200
iteration = 100
epoch = 3
accuracy = 0
while True:
    #if i == iteration or i == 0 or accuracy[0] > 0.95:
    if i == 0 or accuracy[0] > 0.95:
        x, y = None, None
        start += amount
        x, y = get_trainning_data(start, amount)
        i = 0
    print("start", start, "amount", amount, "current iteration", i, "accuracy", accuracy)
    history = model.fit(x, y, batch_size=128, nb_epoch= epoch, shuffle = True)
    accuracy = history.history["acc"]
    model.save_weights("weights/" + str(i%2)+"model.h5")
    i += 1


200
(40192, 30000)
start 300 amount 200 current iteration 0 accuracy 0


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Epoch 1/3
40192/40192 [==============================] - 46s 1ms/step - loss: 7.2006 - acc: 0.0878
Epoch 2/3
40192/40192 [==============================] - 43s 1ms/step - loss: 6.7188 - acc: 0.1035
Epoch 3/3
40192/40192 [==============================] - 43s 1ms/step - loss: 6.4512 - acc: 0.1207
start 300 amount 200 current iteration 1 accuracy [0.08782842356687898, 0.10345342356687898, 0.12074542197452229]
Epoch 1/3
40192/40192 [==============================] - 43s 1ms/step - loss: 6.2843 - acc: 0.1331
Epoch 2/3
40192/40192 [==============================] - 43s 1ms/step - loss: 6.1825 - acc: 0.1435
Epoch 3/3
40192/40192 [==============================] - 43s 1ms/step - loss: 6.1079 - acc: 0.1517
start 300 amount 200 current iteration 2 accuracy [0.1330861863057325, 0.1435111464968153, 0.1517217356687898]
Epoch 1/3
40192/40192 [==============================] - 43s 1ms/step - loss: 6.0630 - acc: 0.1570
Epoch 2/3
40192/40192 [==============================] - 43s 1ms/step - loss: 6.03

40192/40192 [==============================] - 42s 1ms/step - loss: 5.5786 - acc: 0.2184
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.5770 - acc: 0.2196
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.5645 - acc: 0.2203
start 300 amount 200 current iteration 21 accuracy [0.21835191082802546, 0.21964570063694266, 0.2203174761146497]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.5655 - acc: 0.2180
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.5584 - acc: 0.2188
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.5584 - acc: 0.2184
start 300 amount 200 current iteration 22 accuracy [0.21800358280254778, 0.2187748805732484, 0.2184016719745223]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.5455 - acc: 0.2219
Epoch 2/3
40192/40192 [==============================] - 43s 1ms/step - loss: 5.5451 - acc

40192/40192 [==============================] - 42s 1ms/step - loss: 5.3696 - acc: 0.2473
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.3713 - acc: 0.2466
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.3632 - acc: 0.2481
start 300 amount 200 current iteration 41 accuracy [0.24728801751592358, 0.24661624203821655, 0.2480593152866242]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.3778 - acc: 0.2472
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.3662 - acc: 0.2478
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.3640 - acc: 0.2480
start 300 amount 200 current iteration 42 accuracy [0.24716361464968153, 0.24781050955414013, 0.24798467356687898]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.3531 - acc: 0.2492
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 5.3518 - a

...

40192/40192 [==============================] - 42s 1ms/step - loss: 4.8317 - acc: 0.3411
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8243 - acc: 0.3447
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8248 - acc: 0.3428
start 300 amount 200 current iteration 281 accuracy [0.3411375398089172, 0.34467058121019106, 0.3428045382165605]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8160 - acc: 0.3454
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8229 - acc: 0.3454
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8170 - acc: 0.3453
start 300 amount 200 current iteration 282 accuracy [0.34544187898089174, 0.3453672372611465, 0.3453423566878981]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8400 - acc: 0.3429
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8473 - ac

40192/40192 [==============================] - 42s 1ms/step - loss: 4.8198 - acc: 0.3442
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8322 - acc: 0.3428
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8085 - acc: 0.3460
start 300 amount 200 current iteration 301 accuracy [0.34422273089171973, 0.3427547770700637, 0.3460390127388535]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8251 - acc: 0.3429
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8108 - acc: 0.3446
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8316 - acc: 0.3436
start 300 amount 200 current iteration 302 accuracy [0.34290406050955413, 0.34464570063694266, 0.34357583598726116]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8358 - acc: 0.3422
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8082 - 

40192/40192 [==============================] - 43s 1ms/step - loss: 4.8156 - acc: 0.3476
Epoch 2/3
40192/40192 [==============================] - 43s 1ms/step - loss: 4.8218 - acc: 0.3446
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8217 - acc: 0.3481
start 300 amount 200 current iteration 321 accuracy [0.34755672770700635, 0.34457105891719747, 0.34812898089171973]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8200 - acc: 0.3447
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8192 - acc: 0.3467
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8225 - acc: 0.3441
start 300 amount 200 current iteration 322 accuracy [0.3447203423566879, 0.34673566878980894, 0.34414808917197454]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8208 - acc: 0.3461
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8038 -

40192/40192 [==============================] - 42s 1ms/step - loss: 4.8210 - acc: 0.3444
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8190 - acc: 0.3479
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8179 - acc: 0.3466
start 300 amount 200 current iteration 341 accuracy [0.3444466560509554, 0.34785529458598724, 0.34656150477707004]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8356 - acc: 0.3433
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8175 - acc: 0.3472
Epoch 3/3
40192/40192 [==============================] - 43s 1ms/step - loss: 4.8388 - acc: 0.3422
start 300 amount 200 current iteration 342 accuracy [0.34332703025477707, 0.34720839968152867, 0.3421825238853503]
Epoch 1/3
40192/40192 [==============================] - 43s 1ms/step - loss: 4.8264 - acc: 0.3454
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8290 - 

40192/40192 [==============================] - 42s 1ms/step - loss: 4.8377 - acc: 0.3459
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8538 - acc: 0.3385
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8509 - acc: 0.3405
start 300 amount 200 current iteration 361 accuracy [0.34591460987261147, 0.33845043789808915, 0.340515525477707]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8443 - acc: 0.3432
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8473 - acc: 0.3433
Epoch 3/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8524 - acc: 0.3396
start 300 amount 200 current iteration 362 accuracy [0.3431528662420382, 0.34330214968152867, 0.33964470541401276]
Epoch 1/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8503 - acc: 0.3425
Epoch 2/3
40192/40192 [==============================] - 42s 1ms/step - loss: 4.8531 - a

In [ ]:

accuracy

In [ ]:
def get_next(sentence):
    sentence = [sentence]
    sentence = preprocess_sentences(sentence)
    sentence = [" ".join(sentence[0].split(' ')[ -10 :]) ]
    #print(sentence)
    vectors = convert_sentences_to_vectors(sentence, vocab_to_idx)
    x_vec, y_vec = create_trainning_data(vectors, 10, 0, 30000)
    res = model.predict(x_vec)
    vec_y = np.argmax(res, axis=1)
    return idx_to_vocab[vec_y[0]]
lyric = 'why would i do without your smart mouth drawing me in an you kicking me and you are happy'
curr = lyric
result = get_next(curr)
lyric += " " + result
for i in range(50):
    curr = " ".join(curr.split()[1:] + [result])
    result = get_next(curr)
    lyric += " " + result
print(lyric)